In [23]:
import numpy as np
import pandas as pd
import os
import json
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import jsonlines
import pickle

In [22]:
# Load the model from the file
with open('CNN.pkl', 'rb') as file_new:
    model_new = pickle.load(file_new)

In [24]:
model_new.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_50 (Conv2D)              │ (None, 110, 113, 20)   │         5,140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_40 (MaxPooling2D) │ (None, 55, 57, 20)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_50          │ (None, 55, 57, 20)     │            80 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_51 (Conv2D)              │ (None, 55, 57, 22)     │        74,382 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_41 (MaxPooling2D) │ (None, 28, 29, 22)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_51          │ (None, 28, 29, 22)     │            88 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_52 (Conv2D)              │ (None, 28, 29, 26)     │        36,634 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_42 (MaxPooling2D) │ (None, 14, 15, 26)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_52          │ (None, 14, 15, 26)     │           104 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_53 (Conv2D)              │ (None, 14, 15, 38)     │        48,450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_43 (MaxPooling2D) │ (None, 7, 8, 38)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_53          │ (None, 7, 8, 38)       │           152 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_54 (Conv2D)              │ (None, 7, 8, 39)       │        23,751 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_10            │ (None, 4, 4, 39)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_54          │ (None, 4, 4, 39)       │           156 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 4, 4, 39)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 624)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 200)            │       125,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 10)             │         2,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 631,606 (2.41 MB)

 Trainable params: 315,657 (1.20 MB)

 Non-trainable params: 290 (1.13 KB)

 Optimizer params: 315,659 (1.20 MB)

In [16]:
new_file = 'genres_original/jazz/jazz.00002.wav'

In [17]:
height = 128
width = 125
sr=22050
duration=29
audio, _ = librosa.load(new_file, sr=sr, duration=duration)

if len(audio) < sr * duration:
    audio = np.pad(audio, (0, max(0, sr * duration - len(audio))), 'constant')

mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=height, fmax=8000)
mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

mel_spec_db = mel_spec_db[..., np.newaxis] 
mel_spec_db = tf.image.resize(mel_spec_db, (height, width)).numpy()
mel_spec_db = mel_spec_db[np.newaxis, ..., np.newaxis]

In [18]:
genre_dict = {
        0 : "blues",
        1 : "classical",
        2 : "country",
        3 : "disco",
        4 : "hiphop",
        5 : "jazz",
        6 : "metal",
        7 : "pop",
        8 : "reggae",
        9 : "rock",
        }

In [19]:
prediction = model_new.predict(mel_spec_db)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [20]:
predicted_class = np.argmax(prediction, axis=1)
print(f'Predicted class: {genre_dict[predicted_class[0]]}')

Predicted class: jazz
